In [17]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

# 데이터베이스 연결 정보 설정
host = ''
user = ''
password = ''
db = ''
charset = 'utf8'

# 데이터베이스 연결
conn = pymysql.connect(host=host, user=user, password=password, db=db, charset=charset)

# 쿼리 실행 함수
def execute_query(sql):
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

# 오늘 날짜 계산
today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y%m%d')

# 쿼리 실행
query1 = """
    select DATE_FORMAT(FROM_UNIXTIME(o.add_time - 5*60*60), '%Y%m%d') as 'm', 
    sum(if(order_state = 2, 1, 0)) as "Rides", 
    count(order_id) as "Ride Attemp",
    count(distinct o.user_id) as "Riders", sum(order_amount + out_of_area_charge - refund_amount-refund_out_of_area_charge) as "Rev Inc Coupon", 
    sum(pay_amount + out_of_area_charge - refund_amount-refund_out_of_area_charge) as "Rev exclude Coupon", count(distinct bicycle_sn) as "Used Vehicle"
from gbike.rich_orders o 
    left join gbike.rich_user u on o.user_id = u.user_id
where o.add_time between unix_timestamp(now())-(60*60*36) and unix_timestamp(now()) + (60*60*8)
    and o.country_code = 'us'
    and user_type = 0
group by m
order by m asc

""".format(yesterday_str)

query2 = """
    select sum(available_deposit) as "Unpaid Amount-Total", max(user_id) as "Total Users", count(user_id) as "US-Users"
from gbike.rich_user
where country_code = 'us'
"""

query3 = """
    select sum(credit_point = 200) as "No-Ride",
        sum(credit_point = 201) as "One Ride", 
        sum(credit_point = 202) as "Two Rides", 
        sum(credit_point > 201) as "More than Two Rides",    
        sum(driver_license_verification_state = 1) as "driver_license"
from gbike.rich_user
where country_code = 'us'
"""

result1 = execute_query(query1)
result2 = execute_query(query2)
result3 = execute_query(query3)

# 결과 데이터프레임으로 변환
df1 = pd.DataFrame(result1)
df2 = pd.DataFrame(result2)
df3 = pd.DataFrame(result3)

cond = df1['m'] == yesterday_str
df1 = df1[cond].reset_index(drop = True)

# Query2와 Query3를 Query1에 옆으로 붙이기
merged_df = pd.concat([df1, df2, df3], axis=1)

# 엑셀 파일로 저장
file_name = f'미국KPI_{yesterday_str}.xlsx'
writer = pd.ExcelWriter(file_name)
merged_df.to_excel(writer, sheet_name='MergedData', index=False)
writer.save()

# 데이터베이스 연결 종료
conn.close()



In [1]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

# 데이터베이스 연결 정보 설정
host = 'live.st.rds.gbility.io'
user = 'gbikemarketing'
password = 'gbikemkt0514$@#!'
db = 'gbike'
charset = 'utf8'

# 데이터베이스 연결
conn = pymysql.connect(host=host, user=user, password=password, db=db, charset=charset)

# 쿼리 실행 함수
def execute_query(sql):
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

# 오늘 날짜 계산
today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday_str = yesterday.strftime('%Y%m%d')

# 쿼리 실행
query1 = """
    select DATE_FORMAT(FROM_UNIXTIME(o.add_time - 7*60*60), '%Y%m%d') as 'm', 
    sum(if(order_state = 2, 1, 0)) as "Rides", 
    count(order_id) as "Ride Attemp",
    count(distinct o.user_id) as "Riders", sum(order_amount + out_of_area_charge - refund_amount-refund_out_of_area_charge) as "Rev Inc Coupon", 
    sum(pay_amount + out_of_area_charge - refund_amount-refund_out_of_area_charge) as "Rev exclude Coupon", count(distinct bicycle_sn) as "Used Vehicle"
from gbike.rich_orders o 
    left join gbike.rich_user u on o.user_id = u.user_id
where o.add_time between unix_timestamp(now())-(60*60*36) and unix_timestamp(now()) + (60*60*8)
    and o.country_code = 'th'
    and user_type = 0
group by m
order by m asc

""".format(yesterday_str)

query2 = """
    select sum(available_deposit) as "Unpaid Amount-Total", max(user_id) as "Total Users", count(user_id) as "TH-Users"
from gbike.rich_user
where country_code = 'th'
"""

query3 = """
    select sum(credit_point = 200) as "No-Ride",
        sum(credit_point = 201) as "One Ride", 
        sum(credit_point = 202) as "Two Rides", 
        sum(credit_point > 201) as "More than Two Rides",    
        sum(driver_license_verification_state = 1) as "driver_license"
from gbike.rich_user
where country_code = 'th'
"""

result1 = execute_query(query1)
result2 = execute_query(query2)
result3 = execute_query(query3)

# 결과 데이터프레임으로 변환
df1 = pd.DataFrame(result1)
df2 = pd.DataFrame(result2)
df3 = pd.DataFrame(result3)

cond = df1['m'] == yesterday_str
df1 = df1[cond].reset_index(drop = True)

# Query2와 Query3를 Query1에 옆으로 붙이기
merged_df = pd.concat([df1, df2, df3], axis=1)

# 엑셀 파일로 저장
file_name = f'태국KPI_{yesterday_str}.xlsx'
writer = pd.ExcelWriter(file_name)
merged_df.to_excel(writer, sheet_name='MergedData', index=False)
writer.save()

# 데이터베이스 연결 종료
conn.close()